In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import gym
# import d4rl 
import dacmdp
import dacmdp.envs as ce

/nfs/guille/afern/users/shrestaa/installation_files/MinicondaInstallation/envs/modem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/nfs/guille/afern/users/shrestaa/installation_files/MinicondaInstallation/envs/modem/lib/python3.9/site-packages/glfw/__init__.py:906: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
pybullet build time: May 20 2022 19:45:31


In [3]:
# ! pip install scikit-learn

In [4]:
from munch import munchify 

config = munchify({
"envArgs":{'env_name': 'CartPole-cont-v1', 'seed': 0},
"dataArgs": {'buffer_name': 'random', 'buffer_size': 50000, 'load_buffer': False, 'buffer_device': 'gpu', "data_dir":""},
"reprModelArgs": {'repr_model_name': 'OracleDynamicsRepr', 's_multiplyer': 1, 'a_multiplyer': 10, 'repr_dim': 4},
"actionModelArgs": {'action_model_name': 'NNActionModelCuda', 'nn_engine': "torch_pykeops"},
"mdpBuildArgs": {'n_tran_types': 10, 'n_tran_targets': 5, 'penalty_beta': 1.0, 'penalty_type': 'linear', 'rebuild_mdpfcache': False,
                 'save_mdp2cache': False, 'save_folder': '/nfs/hpc/share/shrestaa/storage/dac_storage_22_Q4/mdp_dumps/random_hash'},
"mdpSolveArgs": {'device': 'cuda', 'max_n_backups': 5000, "gamma": 0.99, 'epsilon': 0.0001, 'penalty_beta': 1, "operator": "simple_backup"},
"evalArgs": {'eval_episode_count': 50},
"logArgs": {'no_wandb_logging':True}
})

In [5]:
env = gym.make(config.envArgs.env_name)
data_buffer_train = dacmdp.utils_buffer.generate_or_load_buffer(config, env)

Average Reward of collected trajectories:16.914
Collected buffer!


In [6]:
from dacmdp.core.models_action import NNActionModel, GlobalClusterActionModel, EnsembleActionModel
from dacmdp.core.models_sa_repr import OracleDynamicsRepr

######### Get Action and Repr Models ####################################
nn_action_model = NNActionModel(action_space=env.action_space,
                                   n_actions= config.mdpBuildArgs.n_tran_types,
                                   data_buffer=data_buffer_train,
                                   nn_engine= config.actionModelArgs.nn_engine,
                                   projection_fxn=lambda s: s, 
                                   )
cluster_action_model = GlobalClusterActionModel(action_space=env.action_space,
                                   n_actions= 10,
                                   data_buffer=data_buffer_train)
action_model = EnsembleActionModel(env.action_space,[nn_action_model, cluster_action_model])
config.mdpBuildArgs.n_tran_types = action_model.n_actions

sa_repr_model = OracleDynamicsRepr(env_name = config.envArgs.env_name)
######################################################################################################

Caculating State Representations: : 196it [00:00, 76345.06it/s]
/nfs/guille/afern/users/shrestaa/installation_files/MinicondaInstallation/envs/modem/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [13]:
# sa_repr_model.

In [8]:
import time
from dacmdp.core.dac_core import DACTransitionBatch
from dacmdp.core.dac_build import DACBuildWithActionNames
from dacmdp.core.utils_knn import THelper

######### DACMDP Elastic Build
transitions = DACTransitionBatch(torch.FloatTensor(data_buffer_train.state).clone().detach(),
                                torch.FloatTensor(data_buffer_train.action).clone().detach(),
                                torch.FloatTensor(data_buffer_train.next_state).clone().detach(),
                                torch.FloatTensor(data_buffer_train.reward.reshape(-1)).clone().detach(), 
                                torch.LongTensor((1- data_buffer_train.not_done).reshape(-1)).clone().detach())
print(transitions.rewards.shape)

st = time.time()
elasticAgent = DACBuildWithActionNames( config = config, 
                                    action_space = env.action_space, 
                                    action_model = action_model, 
                                    repr_model = sa_repr_model, 
                                    effective_batch_size= 10000, 
                                    batch_calc_knn_ret_flat_engine = THelper.batch_calc_knn_ret_flat_pykeops
                                    )

elasticAgent.consume_transitions(transitions, verbose = True, batch_size = 10000)
elasticAgent.dacmdp_core.solve(max_n_backups = config.mdpSolveArgs.max_n_backups, 
                               penalty_beta = config.mdpSolveArgs.penalty_beta, 
                               epsilon = config.mdpSolveArgs.epsilon, 
                               gamma = config.mdpSolveArgs.gamma, 
                               operator="simple_backup", 
                               bellman_backup_batch_size=500) 

et = time.time()
print(f"Graph built and solved in {et-st:.2f} Seconds")

torch.Size([50000])
dacmdp_core_defined
Using pre-initialized Action Model BaseActionModel
Using pre-initialized Action Model <dacmdp.core.models_sa_repr.OracleDynamicsRepr object at 0x2b9c08c6c610>
batch_next_states.shape torch.Size([50000, 4])
nn after consumption,  50000


Calculate Candidate Actions: : 5it [00:00, 20.14it/s]
Calculate/Update Datsaet SA Representation: : 5it [00:02,  1.97it/s]
Calculate/Update Candidate Transition SA Representation: : 5it [00:51, 10.40s/it]
Update Transition model of core dacmdp: : 5it [00:00, 28.30it/s]


0 tensor(1.)
500 tensor(0.0067)
1000 tensor(9.9182e-05)
Solved MDP in 1000 Backups
Graph built and solved in 56.12 Seconds


In [9]:
# policy liftup functions
def dummy_lifted_policy(self, s):
    nn_s_idx =  THelper.calc_knn_indices(torch.FloatTensor(s).to(self.device), self.dacmdp_core.S, 1)[0]
    policy_idx = self.dacmdp_core.Pi[nn_s_idx] 
    return self.A_names[nn_s_idx,policy_idx]

In [10]:
from dacmdp.eval.utils_eval import evaluate_on_env
######### Eval
st = time.time()
config.evalArgs.eval_episode_count = 100
avg_rewards, info = evaluate_on_env(env, 
                                    lambda s :dummy_lifted_policy(elasticAgent, s),
                                    eps_count=config.evalArgs.eval_episode_count)
et = time.time()
print("Evaluation compltete", avg_rewards, "Time: ", et-st, "Seconds")
######################################################################################################

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.42it/s]

Evaluation compltete 495.27 Time:  15.595014572143555 Seconds
